# <span style='color:blue'> LAB 7: </span>
# <span style='color:blue'> ADVANCED CNN ARCHITECTURE </span>

In [1]:
import torch
import time
import numpy as np
import seaborn as sns

## <span style='color:red'> MNIST Classification </span>

In [2]:
# Seaborn plot styling
sns.set(style = 'white', font_scale = 2)

## Prepare Data

In [3]:
from torchvision import datasets, transforms

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

## Define Model

In [4]:
from torchvision.models import resnet18

In [5]:
class MNIST_ResNet_Classifier(torch.nn.Module):
    
    def __init__(self, out_channels, kernel_size, stride, padding):
        
        super(MNIST_ResNet_Classifier, self).__init__()

        self.resnet = resnet18()
        
        self.resnet.conv1 = torch.nn.Conv2d(1, out_channels = out_channels, 
                                      kernel_size=(kernel_size, kernel_size), 
                                      stride=(stride, stride), 
                                      padding=(padding, padding), 
                                      bias=False)
        
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, 10)

    def forward(self, x):
        
        return self.resnet(x)

## Define Hyperparameters

In [6]:
MNIST_ResNet_Classifier = MNIST_ResNet_Classifier(out_channels = 64,
                                                   kernel_size = 7,
                                                   stride = 2,
                                                   padding = 3)

# Define learning rate and epochs
learning_rate = 0.001      
epochs = 10

# Batch size for mini-batch gradient
batchsize = 64

# Using Adam as optimizer
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(MNIST_ResNet_Classifier.parameters(), lr=learning_rate)

MNIST_ResNet_Classifier.cuda()

MNIST_ResNet_Classifier(
  (resnet): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

## Identify Tracked values

In [7]:
train_loss_list = []

## Train Model

In [8]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batchsize, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1000, shuffle=False
)

In [9]:
for epoch in range(epochs):
    
    for batch_idx, (train_input, train_target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        pred = MNIST_ResNet_Classifier(train_input.cuda())
        
        loss = loss_func(pred, train_target.cuda())
        
        loss.backward()
        
        optimizer.step()
            
    print("Train Epoch: {} \tLoss: {:.6f}".format(epoch, loss.item()))

Train Epoch: 0 	Loss: 0.103193
Train Epoch: 1 	Loss: 0.009226
Train Epoch: 2 	Loss: 0.048294
Train Epoch: 3 	Loss: 0.011540
Train Epoch: 4 	Loss: 0.082615
Train Epoch: 5 	Loss: 0.038770
Train Epoch: 6 	Loss: 0.009741
Train Epoch: 7 	Loss: 0.021105
Train Epoch: 8 	Loss: 0.001400
Train Epoch: 9 	Loss: 0.021625


## Visualize and Evaluate Model

In [10]:
MNIST_ResNet_Classifier.eval()
test_loss = 0
correct = 0

with torch.no_grad():
    
    for test_input, test_target in test_loader:
        
        output = MNIST_ResNet_Classifier(test_input.cuda())
        
        test_loss += loss_func(output, test_target.cuda()).item()
        
        pred = output.argmax(dim=1, keepdim=True)
        
        correct += pred.eq(test_target.view_as(pred).cuda()).sum().item()

In [11]:
test_loss /= len(test_loader.dataset)

print("Test Accuracy: {}/{} ({:.0f}%)\n".format(correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)))

Test Accuracy: 9905/10000 (99%)

